In [ ]:
import torch
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (13.0, 5.0)

In [ ]:
x_train = torch.rand(100)
x_train = x_train * 20.0 - 10.0

y_train = torch.sin(x_train)

plt.plot(x_train.numpy(), y_train.numpy(), 'o')
plt.title('$y = sin(x)$')

In [ ]:
noise = torch.randn(y_train.shape) / 5.

plt.plot(x_train.numpy(), noise.numpy(), 'o')
plt.axis([-10, 10, -1, 1])
plt.title('Gaussian noise')

In [ ]:
y_train = y_train + noise
plt.plot(x_train.numpy(), y_train.numpy(), 'o')
plt.title('noisy sin(x)')
plt.xlabel('x_train')
plt.ylabel('y_train')

In [ ]:
x_train.unsqueeze_(1)
y_train.unsqueeze_(1)

In [ ]:
x_validation = torch.linspace(-10, 10, 100)
y_validation = torch.sin(x_validation.data)
plt.plot(x_validation.numpy(), y_validation.numpy(), 'o')
plt.title('sin(x)')
plt.xlabel('x_validation')
plt.ylabel('y_validation');

# Архитектура модели

In [ ]:
class SineNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(SineNet, self).__init__()
        self.fc1 = torch.nn.Linear(1, n_hidden_neurons) # Линейная часть
        self.act1 = torch.nn.Sigmoid() # Сигмоидная функция активации
        self.fc2 = torch.nn.Linear(n_hidden_neurons, 1) # Линейная часть
    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        return x

sine_net = SineNet(50) # 50 - колво нейронов в скрытом слое

In [ ]:
# Предсказание на рандомных весовых кф.
def predict(net, x, y):
    y_pred = net.forward(x)

    plt.plot(x.numpy(), y.numpy(), 'o', label='Groud truth')
    plt.plot(x.numpy(), y_pred.data.numpy(), 'o', c='r', label='Prediction');
    plt.legend(loc='upper left')
    plt.xlabel('$x$')
    plt.ylabel('$y$')

predict(sine_net, x_validation, y_validation)

# Обучение нейросети

In [ ]:
optimizer = tch.optim.Adam(sine_net.parameters(),lr=0.01)
# Внутрь метода передаются веса которые содержатся в атрибуте parameters

In [ ]:
# Функция потерь
def loss(pred,target):
    squares = (target - pred)**2
    return squares.mean()

In [ ]:
for epoch_index in range(2000): # 2000 эпох
    optimizer.zero_grad() # Зануляем градиенты

    y_pred = sine_net.forward(x_train) # Построили сеть с весовыми кф.
    loss_val = loss(y_pred, y_train) # Посчитали функцию потреь

    loss_val.backward() # Нашли градиенту по весам  функции потерь

    optimizer.step() # Сделали спуск

predict(sine_net, x_validation, y_validation)

# Нейросеть для классификации

In [ ]:
import torch
import random
import numpy as np

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:
import sklearn.datasets
wine = sklearn.datasets.load_wine()
wine.data.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(wine.data[:, :2], 
    wine.target, 
    test_size=0.3, 
    shuffle=True)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
class WineNet(tch.nn.Module):
    def __init__(self,n_hidden_neurons):
        super(WineNet, self).__init__()
        self.fc1 = tch.nn.Linear(2,n_hidden_neurons)
        self.act1 = tch.nn.Sigmoid()
        self.fc2 = tch.nn.Linear(n_hidden_neurons,n_hidden_neurons)
        self.act2 = tch.nn.Sigmoid()
        self.fc3 = tch.nn.Linear(n_hidden_neurons,3)
        self.sm = tch.nn.Softmax(dim=1)
   
    def forward(self,x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x)
        return x
        
        
    def inference(self,x):
        x = self.forward(x)
        x = self.sm(x)
        return x
    
wine_net = WineNet(5)

In [ ]:
loss = tch.nn.CrossEntropyLoss()

optimizer = tch.optim.Adam(wine_net.parameters(),lr=1.0e-3)

In [ ]:
np.random.permutation(10) # Для рандомного смешивания 

In [ ]:
batch = 10

for epoch in range(5000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0,len(X_train),batch):
        optimizer.zero_grad()
        batch_index = order[start_index:start_index+batch]
        
        X_train_batch = X_train[batch_index]
        y_train_batch = y_train[batch_index]
        
        pred = wine_net.forward(X_train_batch)
        
        loss_pred = loss(pred,y_train_batch)
        
        loss_pred.backward()
        
        optimizer.step()
        
    if epoch % 100 == 0:
        test_pred = wine_net.forward(X_test)
        test_pred = test_pred.argmax(dim=1)
        print((test_pred == y_test).float().mean())